###  Importing Necessary Libraries

In [5]:
!pip3 install Pyspellchecker
!pip3 install kaggle
!pip install gensim
%pip install pyspellchecker
%pip install scrapetube
%pip install google-api-python-client
%pip install pandas
%pip install langdetect
%pip install nltk
%pip install scikit-learn

!pip list


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached gensim-4.3.3-cp38-cp38-macosx_10_9_x86_64.whl.metadata (8.3 kB)
  Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl.metadata (53 kB)
  Using cached smart_open-7.0.5-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.16.0-cp38-cp38-macosx_10_9_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 18.5 MB/s eta 0:00:00a 0:00:01
   ━━━

In [1]:
import gensim
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import json 
import os


ModuleNotFoundError: No module named 'gensim'

### Initializing NLTK Resources and Preprocessing Tools

In [4]:
#download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

#initialize stemmer, lemmatizer, and spell checker
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
spell = SpellChecker()

[nltk_data] Downloading package punkt to /Users/siva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/siva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/siva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/siva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


###  Loading the Dataset From Kaggle

In [5]:
import os
import zipfile

#define the path where you want to download the dataset
download_path = "/Users/siva/Downloads/kaggle_datasets"

#download the dataset using Kaggle API
os.system(f"kaggle datasets download -d sivameenakshi/Online-Football-Comments -p {download_path}")

#define the dataset name
dataset_name = "Online-Football-Comments"

#define the path for the zip file
zip_path = os.path.join(download_path, f"{dataset_name}.zip")

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/Users/siva/Downloads/kaggle_datasets")  #extract to 'dataset' folder
    print("Dataset unzipped successfully!")
else:
    print("File not found. Please check if the dataset was downloaded correctly.")



/Users/siva/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/sivameenakshi/Online-Football-Comments
License(s): unknown
Online-Football-Comments.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset unzipped successfully!


###  Combining Reddit CSV files from Dataset

In [6]:
import pandas as pd
import glob

#select all reddit CSV files in the directory
file_path = "/Users/siva/Downloads/kaggle_datasets/OnlineFootballCommentsDataset/reddit*.csv"

#use glob to find all CSV files matching the pattern
all_files = glob.glob(file_path)

#initialize an empty list to hold DataFrames
dataframes = []

#loop through the list of files and read each one
for filename in all_files:
    df = pd.read_csv(filename)  
    #append the dataframes to the list
    dataframes.append(df)        

#add all dataFrames into a single dataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

#create the path for the combined CSV file
combined_file_path = os.path.join("/Users/siva/Downloads/kaggle_datasets/OnlineFootballCommentsDataset/", "combined_reddit_data.csv")

#saving the combined dataFrame to a new CSV file in the same folder
combined_df.to_csv(combined_file_path, index=False)

#display the first few rows of the combined dataFrame
print(combined_df.head())

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
                                       Post Title  \
0  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
1  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
2  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
3  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
4  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    

                                        Comment Body              Author  
0         As a pre-Messi MIA fan this makes me happy         EctoRiddler  
1        From last in the league to first is madness         TaxHedgehog  
2  Messi comes and gives Miami their first 2 trop...              tusou4  
3  46 trophies in a single career span is madness...  SorasDestinyIsland  
4  Hopefully Messi can reach 50 titles in his career     NguyenBangGiang  


#### Reddit Comment Cleaning

In [26]:
#remove comments authored by 'AutoModerator'
combined_df = df[df['Author'] != 'AutoModerator']
print(combined_df.head())

                                       Post Title  \
0  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
1  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
2  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
3  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    
4  [Inter Miami] - SUPPORTERS’ SHIELD CHAMPIONS!    

                                        Comment Body              Author  
0         As a pre-Messi MIA fan this makes me happy         EctoRiddler  
1        From last in the league to first is madness         TaxHedgehog  
2  Messi comes and gives Miami their first 2 trop...              tusou4  
3  46 trophies in a single career span is madness...  SorasDestinyIsland  
4  Hopefully Messi can reach 50 titles in his career     NguyenBangGiang  


#### Removing Non-English Comments From Reddit Comments

In [ ]:
import pandas as pd
from langdetect import detect, DetectorFactory

#setting langdetect seed for reproducibility
DetectorFactory.seed = 0

# Load the CSV file
df = pd.read_csv("/Users/siva/Downloads/kaggle_datasets/OnlineFootballCommentsDataset/combined_reddit_data.csv")

# Function to detect language
def is_english(comment):
    try:
        # Attempt to detect the language
        return detect(comment) == 'en'
    except Exception as e:
        # If an error occurs (e.g., due to empty comments), return False
        return False

# Filter the DataFrame to keep only English comments
df = df[df['Comment Body'].apply(is_english)]

# Display the first few rows to confirm changes
print(df.head())


###  Preprocessing Reddit Text Data (Tokenization, Stemming, Lemmatization, Removing Stopwords)

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer


def preprocess_text(text):
    #check if the text is None
    if text is None:
        return ''

    #tokenization and lowercasing
    tokens = word_tokenize(text.lower())

    #remove punctuation and non-alphabetic characters
    tokens = [t for t in tokens if t.isalpha()]

    #stemming and Lemmatization
    stemmed = [stemmer.stem(t) for t in tokens if t is not None]
    lemmatized = [lemmatizer.lemmatize(t, pos='v') for t in stemmed if t is not None]

    #remove stopwords
    stop_words = set(stopwords.words('english'))
    #custom stopwords added
    custom_stopwords = {'a', 'an', 'the', 'in', 'on', 'at', 'by', 'for', 'and', 'but', 'or', 'so', 'he', 'she', 'it', 'they', 'we'}
    stop_words = stop_words.union(custom_stopwords)

    filtered_words = [w for w in lemmatized if not w in stop_words]

    return ' '.join(filtered_words)

#apply the preprocessing function to the DataFrame
df['Comment Body'] = df['Comment Body'].apply(preprocess_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/siva/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### Initializing and Applying TF-IDF Vectorizer for Reddit

In [31]:
#load the preprocessed comments and handle any missing values
redditcomments = df['Comment Body'].fillna('')  # Replace NaNs with empty strings
#initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(df['Comment Body'])

#### Analyzing TF-IDF Reddit Results

In [33]:
#retrieve feature names (words) from the vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

#extract and analyze TF-IDF scores for the first document (comment)
first_document_vector = tfidf_matrix[0]

#convert the TF-IDF results for the first document into a readable DataFrame
df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])

df_tfidf = df_tfidf.sort_values(by=["tfidf"], ascending=False)

print(df_tfidf.head(20))

              tfidf
mia        0.522515
pre        0.421937
happy      0.343468
makes      0.318962
fan        0.311722
messi      0.302898
me         0.255511
as         0.217387
this       0.166672
čeferin    0.000000
007        0.000000
îndoiesc   0.000000
öppnar     0.000000
östersund  0.000000
över       0.000000
özil       0.000000
özils      0.000000
ødegaard   0.000000
últimos    0.000000
über       0.000000


### Reddit Bag of Words


In [34]:
from sklearn.feature_extraction.text import CountVectorizer

#initialize CountVectorizer
count_vectorizer = CountVectorizer()

#fit and transform the comments
bow_matrix = count_vectorizer.fit_transform(redditcomments)

#get feature names (words)
bow_feature_names = count_vectorizer.get_feature_names_out()

print('Bag of Words matrix shape:', bow_matrix.shape)
print('Sample BoW features:', bow_feature_names[:20])

Bag of Words matrix shape: (1783827, 21162)
Sample BoW features: ['00' '000' '007' '00s' '01' '02' '03' '04' '05' '06' '07' '08' '09'
 '09zjqrw' '0bc' '0rdoyykorwk' '0rjjsnfugjhnnw9s' '10' '100' '1000']


### Reddit Word2Vec

In [17]:
import gensim
from gensim.models import Word2Vec

#tokenize the preprocessed comments
tokenized_comments = [comment.split() for comment in redditcomments]

#train a Word2Vec model
word2vec_model = Word2Vec(tokenized_comments, vector_size=100, window=5, min_count=1, workers=4)

#view Word2Vec results
print('Vocabulary size:', len(word2vec_model.wv.key_to_index))
print('Vector for a sample word:', word2vec_model.wv['premier'])  #sample word from the data

ModuleNotFoundError: No module named 'gensim'

#### Acessing Youtube Comment Dataset

In [20]:
#load the CSV file
df = pd.read_csv('/Users/siva/Downloads/kaggle_datasets/OnlineFootballCommentsDataset/all_youtube_comments.csv')

print(df.head())

       VideoID                                            Comment  \
0  bmR98lYurtc  Forest don’t play premier league football, the...   
1  bmR98lYurtc                              Hope bren&#39;s okay.   
2  bmR98lYurtc  The referee is such a weird guy that he needs ...   
3  bmR98lYurtc  Gotta be wary of offences &amp; çards............   
4  bmR98lYurtc                               Semangat untuk juara   

               Author             Timestamp  
0    @JackSouth-gm6qr  2023-12-16T21:04:02Z  
1            @alae709  2023-12-16T21:03:18Z  
2        @junodoh6358  2023-12-16T20:54:08Z  
3  @williamwilkes9873  2023-12-16T20:25:49Z  
4     @joypradana9237  2023-12-16T19:38:27Z  


#### Youtube HTML Entity Conversion 

In [22]:
#remove all extra spaces and newlines:
df['Comment'] = df['Comment'].apply(lambda x: ' '.join(x.split()))
#replace &amp; with &
df['Comment'] = df['Comment'].str.replace('&amp;', '&', regex=False)
#replace &#39; with '
df['Comment'] = df['Comment'].str.replace('&#39;', "'", regex=False)
#replace &quot; with "
df['Comment'] = df['Comment'].str.replace('&quot;', '"', regex=False) 
#replace &lt; with <
df['Comment'] = df['Comment'].str.replace('&lt;', '<', regex=False) 
#replace &gt; with >   
df['Comment'] = df['Comment'].str.replace('&gt;', '>', regex=False)    

#display the first few rows to confirm changes
print(df.head())

       VideoID                                            Comment  \
0  bmR98lYurtc  Forest don’t play premier league football, the...   
1  bmR98lYurtc                                  Hope bren's okay.   
2  bmR98lYurtc  The referee is such a weird guy that he needs ...   
3  bmR98lYurtc     Gotta be wary of offences & çards.............   
4  bmR98lYurtc                               Semangat untuk juara   

               Author             Timestamp  
0    @JackSouth-gm6qr  2023-12-16T21:04:02Z  
1            @alae709  2023-12-16T21:03:18Z  
2        @junodoh6358  2023-12-16T20:54:08Z  
3  @williamwilkes9873  2023-12-16T20:25:49Z  
4     @joypradana9237  2023-12-16T19:38:27Z  


#### Removing Non-English Comments From Youtube Comments

In [33]:
import pandas as pd
from langdetect import detect, DetectorFactory

#setting langdetect seed for reproducibility
DetectorFactory.seed = 0

# Load the CSV file
df = pd.read_csv("/Users/siva/Downloads/kaggle_datasets/OnlineFootballCommentsDataset/all_youtube_comments.csv")

# Function to detect language
def is_english(comment):
    try:
        # Attempt to detect the language
        return detect(comment) == 'en'
    except Exception as e:
        # If an error occurs (e.g., due to empty comments), return False
        return False

# Filter the DataFrame to keep only English comments
all_youtube_comments= df[df['Comment'].apply(is_english)]

# Display the first few rows to confirm changes
print(all_youtube_comments.head())


        VideoID                                            Comment  \
0   bmR98lYurtc  Forest don’t play premier league football, the...   
2   bmR98lYurtc  The referee is such a weird guy that he needs ...   
3   bmR98lYurtc  Gotta be wary of offences &amp; çards............   
7   bmR98lYurtc                Spurs should buy Onana from Everton   
10  bmR98lYurtc  Interpol is looking for Awakenbeerus, the guy ...   

                  Author             Timestamp  
0       @JackSouth-gm6qr  2023-12-16T21:04:02Z  
2           @junodoh6358  2023-12-16T20:54:08Z  
3     @williamwilkes9873  2023-12-16T20:25:49Z  
7         @operationbuzz  2023-12-16T17:51:54Z  
10  @Shalaka-Tambura0827  2023-12-16T17:24:28Z  


###  Preprocessing Youtube Text Data (Tokenization, Stemming, Lemmatization, Removing Stopwords)

In [32]:
def preprocess_text(text):
    #check if the text is None
    if text is None:
        return ''

    #tokenization and lowercasing
    tokens = word_tokenize(text.lower())

    #remove punctuation and non-alphabetic characters
    tokens = [t for t in tokens if t.isalpha()]

    #stemming and Lemmatization
    stemmed = [stemmer.stem(t) for t in tokens if t is not None]
    lemmatized = [lemmatizer.lemmatize(t, pos='v') for t in stemmed if t is not None]

    #remove stopwords
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'a', 'an', 'the', 'in', 'on', 'at', 'by', 'for', 'and', 'but', 'or', 'so', 'he', 'she', 'it', 'they', 'we'}
    stop_words = stop_words.union(custom_stopwords)

    filtered_words = [w for w in lemmatized if not w in stop_words]

    return ' '.join(filtered_words)

#apply the preprocessing function to the DataFrame
all_youtube_comments = df['Comment'].apply(preprocess_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/siva/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/usr/local/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### Initializing and Applying TF-IDF Vectorizer for Youtube

In [25]:
#load the preprocessed comments and handle any missing values
ytcomments = df['Comment'].fillna('')  # Replace NaNs with empty strings
#initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(df['Comment'])

#### Analyzing TF-IDF Youtube Results

In [27]:
#retrieve feature names (words) from the vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

#extract and analyze TF-IDF scores for the first document (comment)
first_document_vector = tfidf_matrix[0]

#convert the TF-IDF results for the first document into a readable DataFrame
df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])

df_tfidf = df_tfidf.sort_values(by=["tfidf"], ascending=False)

print(df_tfidf.head(15))

             tfidf
don       0.405909
they      0.335499
cooper    0.331728
rid       0.326134
forest    0.280592
ll        0.257933
down      0.251048
premier   0.225616
football  0.211802
go        0.209041
get       0.197485
play      0.192182
league    0.185658
if        0.178970
of        0.126853


#### Youtube Bag of Words

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

#initialize CountVectorizer
count_vectorizer = CountVectorizer()

#fit and transform the comments
bow_matrix = count_vectorizer.fit_transform(df['Comment'])

#get feature names (words)
bow_feature_names = count_vectorizer.get_feature_names_out()

print('Bag of Words matrix shape:', bow_matrix.shape)
print('Sample BoW features:', bow_feature_names[:10])

Bag of Words matrix shape: (45422, 23474)
Sample BoW features: ['00' '000' '00000' '000000001s' '0001' '000ks' '000th' '001' '008'
 '00h00']


#### Youtube Word2Vec

In [35]:
from gensim.models import Word2Vec

#tokenize the preprocessed comments
tokenized_comments = [comment.split() for comment in all_youtube_comments]

#train a Word2Vec model
word2vec_model = Word2Vec(tokenized_comments, vector_size=100, window=5, min_count=1, workers=4)

#view Word2Vec results
print('Vocabulary size:', len(word2vec_model.wv.key_to_index))
print('Vector for a sample word:', word2vec_model.wv['premier'])  #sample word from the data

ModuleNotFoundError: No module named 'gensim'